In [2]:
# Load model directly
from huggingface_hub import snapshot_download

snapshot_download(repo_id="Qwen/Qwen3-Embedding-0.6B", cache_dir='/data/LLMs/')

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.71k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/data/LLMs/models--Qwen--Qwen3-Embedding-0.6B/snapshots/744169034862c8eec56628663995004342e4e449'